# Пакеты

In [13]:
from typing import List, Any
!pip install augmentex

In [50]:
import pandas as pd
from enum import Enum
from augmentex import CharAug
import random

# Загрузка данных из файла

In [2]:
df = pd.read_csv("places_address_elements.csv")
df = df.fillna('')

In [4]:
df

,region_name,region_type_name,region_type_name_short,area_name,area_type_name,area_type_name_short,city_name,city_type_name,city_type_name_short,territory_name,territory_type_name,territory_type_name_short,street_name,street_type_name,street_type_name_short
0,Адыгея,Республика,Респ,,,,,,,,,,,,
1,Башкортостан,Республика,Респ,,,,,,,,,,,,
2,Алтай,Республика,Респ,,,,,,,,,,,,
3,Кабардино-Балкарская,Республика,Респ,,,,,,,,,,,,
4,Калмыкия,Республика,Респ,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519314,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Малиновая,Улица,ул.
1519315,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Персиковая,Улица,ул.
1519316,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Яблоневая,Улица,ул.
1519317,Краснодарский,Край,край,Тихорецкий,Район,р-н,Тихорецк,Город,г,Западный-4,Территория СНТ,тер. СНТ,Садовая,Улица,ул.


# Вспомогательные классы
## Словарь ElementType 

In [5]:

class ElementType(Enum):
    OTHER = 0
    REGION=1
    REGION_TYPE=2
    AREA=3
    AREA_TYPE=4
    TERRITORY=5
    TERRITORY_TYPE=6
    CITY=7
    CITY_TYPE=8
    STREET=9
    STREET_TYPE=10


## Класс TypedWord
Проставим каждому токену тип. Для этого введем класс TypedWord

In [6]:
class TypedWord:
    word = ''
    type = ElementType.OTHER
    def __init__(self, word:str, type:ElementType):
        self.word = word
        self.type = type
    
    def __repr__(self):
        return self.word + ":" + self.type.name

# Аугментация

## Представление набора адресо-образующих элементов в виде списка TypedWord

In [58]:
def to_typed_word_list(input , elementType :ElementType  ) -> list:
    words = list(filter(('').__ne__, input.split(' ')))
    res = list()
    for word in words:
        res.append(TypedWord(word, elementType))
    return res


def convert_address_elements(elements: dict, type_first_proba=0.75, comma_sep_proba = 0.75):
    
    res: list[TypedWord] = []
    if type_first_proba>random.random():
        res+= to_typed_word_list(elements['region_type_name'], ElementType.REGION_TYPE)
        res+= to_typed_word_list(elements['region_name'], ElementType.REGION)
    else:
        res+= (to_typed_word_list(elements['region_name'], ElementType.REGION))
        res+= to_typed_word_list(elements['region_type_name'], ElementType.REGION_TYPE)
    
    if comma_sep_proba>random.random():
        last_typed_word = res[-1]
        last_typed_word.word = last_typed_word.word + ','

    if type_first_proba>random.random():
        res+= to_typed_word_list(elements['area_type_name'], ElementType.AREA_TYPE)
        res+= to_typed_word_list(elements['area_name'], ElementType.AREA)
    else:
        res+= to_typed_word_list(elements['area_name'], ElementType.AREA)
        res+= to_typed_word_list(elements['area_type_name'], ElementType.AREA_TYPE)

    if comma_sep_proba>random.random():
        last_typed_word = res[-1]
        last_typed_word.word = last_typed_word.word + ','

    if type_first_proba>random.random():
        res+= to_typed_word_list(elements['city_type_name'], ElementType.CITY_TYPE)
        res+= to_typed_word_list(elements['city_name'], ElementType.CITY)
    else:
        res+= to_typed_word_list(elements['city_name'], ElementType.CITY)
        res+= to_typed_word_list(elements['city_type_name'], ElementType.CITY_TYPE)

    if comma_sep_proba>random.random():
        last_typed_word = res[-1]
        last_typed_word.word = last_typed_word.word + ','
        
    if type_first_proba>random.random():
        res+= to_typed_word_list(elements['territory_type_name'], ElementType.TERRITORY_TYPE)
        res+= to_typed_word_list(elements['territory_name'], ElementType.TERRITORY)
    else:
        res+= to_typed_word_list(elements['territory_name'], ElementType.TERRITORY)
        res+= to_typed_word_list(elements['territory_type_name'], ElementType.TERRITORY_TYPE)

    if comma_sep_proba>random.random():
        last_typed_word = res[-1]
        last_typed_word.word = last_typed_word.word + ','    
        
    if type_first_proba>random.random():
        res+= to_typed_word_list(elements['street_type_name'], ElementType.STREET_TYPE)
        res+= to_typed_word_list(elements['street_name'], ElementType.STREET)
    else:
        res+= to_typed_word_list(elements['street_name'], ElementType.STREET)
        res+= to_typed_word_list(elements['street_type_name'], ElementType.STREET_TYPE)

    if comma_sep_proba>random.random():
        last_typed_word = res[-1]
        last_typed_word.word = last_typed_word.word + ','

    return res


elements = df.sample().to_dict('records')
convert_address_elements(elements[0])

[Ленинградская:REGION,
 Область:REGION_TYPE,
 Район:AREA_TYPE,
 Приозерский,:AREA,
 Поселок:CITY_TYPE,
 Сосново,,:CITY,
 Переулок:STREET_TYPE,
 Лесной,:STREET]

In [59]:
sample_typed_words = convert_address_elements(df.sample().to_dict('records')[0])
sample_typed_words

[Область:REGION_TYPE,
 Московская,:REGION,
 Город:AREA_TYPE,
 Дмитров,:AREA,
 Рабочий:CITY_TYPE,
 поселок:CITY_TYPE,
 Некрасовский:CITY,
 Территория:TERRITORY_TYPE,
 Лесная:TERRITORY,
 поляна,:TERRITORY]

In [48]:
char_aug = CharAug(
    unit_prob=0.05, # Percentage of the phrase to which augmentations will be applied
    min_aug=0, # Minimum number of augmentations
    max_aug=3, # Maximum number of augmentations
    mult_num=3, # Maximum number of repetitions of characters (only for the multiply method)
    random_seed=42,
    lang="rus", # supports: "rus", "eng"
    platform="pc", # supports: "pc", "mobile"
)


In [49]:
text = 'Автодорога Москва-Казань'
char_aug.augment(text=text)

'Автодорога Москва-Казыань'

Обратная задача. Представление списка TypedWords в виде списка слов или списка типов адресо-образующих элементов

In [60]:
def typed_words_to_words(input:list):
    words = []
    for element in input:
        words.append(element.word)
    return words    

In [61]:
def typed_words_to_type_codes(input:list):
    type_codes = []
    for element in input:
        type_codes.append(element.type.value)
    return type_codes    

# Генерация нового датасета
## Новые колонки - tokens и classes

In [62]:
df['typed_words'] = df.apply(convert_address_elements, axis=1)
df['tokens'] = df.apply(lambda row: typed_words_to_words(row.typed_words),  axis=1)
df['classes'] = df.apply(lambda row: typed_words_to_type_codes(row.typed_words),  axis=1)

## Конструирование нового dataframe 

In [18]:
df_token_with_classes = df[['tokens','classes']]

## Сохранение на диск

In [19]:
df_token_with_classes.to_pickle('./df_token_with_classes.pkl')

In [20]:
!zip archive_pkl.zip ./df_token_with_classes.pkl  

  adding: df_token_with_classes.pkl (deflated 90%)
